In [47]:
import sys
import time
import datetime
import json
import constants
import requests
import psycopg

from riotwatcher import LolWatcher, ValWatcher, ApiError, RiotWatcher

import match_fetcher


In [48]:
YrdenPass = constants.db_password
host_ip = constants.db_ip

In [49]:
lol_watcher = LolWatcher(constants.API_KEY_SERVICE)
riot_watcher = RiotWatcher(constants.API_KEY_SERVICE)
api_key = constants.API_KEY_SERVICE

In [50]:
YrdenPass = constants.db_password
host_ip = constants.db_ip
#Connect to Yrden DB
#Private args are executed and then deleted for safety
db_connection = f'dbname=YRDEN user=postgres password={YrdenPass} host={host_ip}'

conn = psycopg.connect(db_connection)

#Creates a cursor for db operations
cur = conn.cursor()

In [51]:
cur.execute('SELECT riot_puuid FROM "YRDEN".yrden_people WHERE summoner_name is NOT NULL AND riot_puuid is NOT NULL')
result = cur.fetchall()
res = [result[0] for result in result]
res

['oFdCaHSdLE-umAd_AUTwskzbTOEPp-CU7-73Gv7wTpF2TWKw18JTx8F86EzUTMf6goBR9ZVoAwFUWA',
 'Hd3lUYJgb7DENibojGOd_jOBjNUojDepDeCs6OLxHF80NkRdwr9UA_BJK5qQUD7h6wacRJ42sb8uPA',
 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
 'verjeqL6V6X4yDSQ6frKQSdh_u9OjwMfp9YJ3jetwkcNj-tyzov3zduVtY-SEVUGAQ_hz4lLE7URsA',
 'QQJnzlpLTSBZvRvKUe4MDfJiFGAf3G-IDy0T96h7gCB5T9BN40K9tBzpg2SnFP55oxdqI8LTUQZGeg',
 '0CpS1bKgKnH7IKuMVnpeOpyLFIj3J78S1jpxZmOx3EZq4jNp4LyIChKBAbfwpnh0of9ooKp7h8VaPQ',
 'Xhi01hd4YPU92OGkFKSsqEnTtVwTD_lERDpFttdWrpKSPBBZ698GvDYGkYEeq31DVz67O3oNNlEgSw',
 'HmOcVn8rE4ArEVltljRl4pWndPLIPRf9GTG3WA_vyrJXJ7fQLVGXqZ1yYVgBTXxn3-x2ACphfh9Syw',
 'VzvO1bwgi0OQCnxU96D0R-wrrCp4aM9qbA4R0lSJ9ZMPD3gRVSsfxRI5x7KUJdlY0205lG862-tywQ',
 'nQd7Ylk472T9NtwuS-hhuLDcxoUg71HK60rRIvJY1b9YMylIjeQKRVzA6p3L7RTm7OMfNISOQMR-5Q',
 '84_xB7Ms1eQbyvGb6w7SKbxPm2AtlUUfCHgqF5WrgKNhpeppB8uzrtS6osCVQ5afvmVWAcnmXZWROg',
 'a62cDaHm9nna2bWRLfiDZ2foba7f-aK663hWCJiEr8M1p9MRM9LnXO4ZBBO3j9X3oYUqvuspiKJt2w',
 '4F

In [30]:
cur.execute(''' SELECT RIOT_PUUID FROM "YRDEN".YRDEN_PEOPLE WHERE RIOT_PUUID IS NOT NULL;''')
puuids = cur.fetchall()

lol_watcher.summoner.by_puuid('na1', puuids[2])

{'id': 'kFIKtjYdraYAo5B2Stz2H9G8Pm_tER1-usRFbQMSlUJ92wc',
 'accountId': '8kvriQI1e0E1m6ZsV6fw8HWs5QYPr088Cvy9DU_EPre_y3I',
 'puuid': 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
 'name': 'Antzi',
 'profileIconId': 4407,
 'revisionDate': 1702876133065,
 'summonerLevel': 830}

In [29]:
summoner_ids = {}
cur.execute('''SELECT summoner_id FROM "YRDEN".yrden_people WHERE summoner_id IS NOT NULL;''')
result = cur.fetchall()
result


[('NIjHCSkwyUmf5e7g54OhPRci_L9vPh4gs54eEhIF4BBdihs',),
 ('9vOEt6pNlVxMBoaOwzqum4hkIeGSHL3MxCdIIvhCi26RPPE',),
 ('kFIKtjYdraYAo5B2Stz2H9G8Pm_tER1-usRFbQMSlUJ92wc',),
 ('hG0EMEDSLadFMgiqkRzeO73TlHrE9U6hkXVM1hG-f6tDkug',),
 ('qZDiiKmSUHCv7mAaZi0uLoiCuJS7MwJeR7LOsM32FOLeJk0',),
 ('1g5Gu9JPJ1SY7xe0F5D0SN87VCzVOo_1O80BZVHF7alY5bQ',),
 ('lZ5x4X43CIY2KEy2TVnhL8CKzia0hZXDW2mfULy3CUjTAPs',),
 ('3ESf1Y_PigqlszeVJMs2Hyy2crhTOGY0Md47fBXV3NeKHmU',),
 ('9M6pduy8wejqoXUpz17ZszkvTJv5BGUR9_A4pcZFbxi6MT8',),
 ('Wvn0dFkr7o8s4Udrh-P4bonOcTINYrOizVhlZufJaJQ-E4c',),
 ('VuTUVP490aBM6hTJX0BVBq9JhlbOlH8HtfjZKOKBX6NBc6U',),
 ('EAZqsPSSQkUUPZ8cnTcSZhDwIEWnWn9KvcT-7f8dGAlzdQM',),
 ('TQBQJfHgCl_cYkR3B1lqC1wdvPW6C1fiufgiiQz-0_WHLPQ',),
 ('jTaF-pVnbaPRouuvCPGkG29xcHzPQ54mfAnp-glTRcJ8xJM',)]

In [31]:
lol_watcher.champion_mastery.by_summoner('na1', result[5][0])

HTTPError: 404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/1g5Gu9JPJ1SY7xe0F5D0SN87VCzVOo_1O80BZVHF7alY5bQ

In [27]:
#Creating Champion Mastery Table

cur.execute ('''DROP TABLE
              "YRDEN".lol_champ_mastery''')
cur.execute('''CREATE TABLE "YRDEN".lol_champ_mastery
            (puuid varchar , 
            summonerId varchar,
            championId int , 
            championLevel int ,
            championPoints int ,
            championPointsUntilNextLevel int,
            championPointsSinceLastLevel int,
            chestGranted boolean,
            lastPlayTime timestamp,
            tokensEarned int
            )
            ;
            ''')
conn.commit()



In [ ]:
result[1]
res = [result[0] for result in result]

In [20]:
result

NameError: name 'result' is not defined

In [ ]:
res

In [ ]:
"""
for n in range(len(result)):
    vals = lol_watcher.champion_mastery.by_summoner('na1', result[n])
    new_vals = {}
    for i in range(len(vals)):
        new_vals.update({'puuid' : vals[n]['puuid'],
                                'summonerId' : vals[n]['summonerId'],
                                 'championId' : vals[n]['championId'],
                                 'championLevel' : vals[n]['championLevel'],
                                 'championPoints' :  vals[n]['championPoints'],
                                 'championPointsUntilNextLevel' :  vals[n]['championPointsUntilNextLevel'],
                                 'championPoinstSinceLastLevel' :  vals[n]['championPointsSinceLastLevel'],
                                 'chestGranted' :  vals[n]['chestGranted'],
                                 'lastPlayTime' :  vals[n]['lastPlayTime'],
                                 'tokensEarned' :  vals[n]['tokensEarned']
                                })
        print(new_vals)
        """

In [ ]:
timestamp_in_milliseconds = 1700529785000
timestamp_in_seconds = timestamp_in_milliseconds / 1000

dt_object = datetime.datetime.fromtimestamp(timestamp_in_seconds)
print(dt_object)
print(timestamp_in_seconds)


In [ ]:
lol_watcher.champion_mastery.by_summoner('na1', 'pehrpiJgcFnMT07UJyGZQ2TESuv0ndIBChe41GVtKw6ehhE')[0]['lastPlayTime']/1000

In [ ]:
print(datetime.datetime.fromtimestamp(lol_watcher.champion_mastery.by_summoner('na1', 'pehrpiJgcFnMT07UJyGZQ2TESuv0ndIBChe41GVtKw6ehhE')[0]['lastPlayTime']/1000))

In [22]:
cur.execute('SELECT riot_puuid FROM "YRDEN".yrden_people WHERE summoner_name is NOT NULL AND riot_puuid is NOT NULL')
result = cur.fetchall()
res = [result[0] for result in result]
res

['oFdCaHSdLE-umAd_AUTwskzbTOEPp-CU7-73Gv7wTpF2TWKw18JTx8F86EzUTMf6goBR9ZVoAwFUWA',
 'Hd3lUYJgb7DENibojGOd_jOBjNUojDepDeCs6OLxHF80NkRdwr9UA_BJK5qQUD7h6wacRJ42sb8uPA',
 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
 'verjeqL6V6X4yDSQ6frKQSdh_u9OjwMfp9YJ3jetwkcNj-tyzov3zduVtY-SEVUGAQ_hz4lLE7URsA',
 'QQJnzlpLTSBZvRvKUe4MDfJiFGAf3G-IDy0T96h7gCB5T9BN40K9tBzpg2SnFP55oxdqI8LTUQZGeg',
 '0CpS1bKgKnH7IKuMVnpeOpyLFIj3J78S1jpxZmOx3EZq4jNp4LyIChKBAbfwpnh0of9ooKp7h8VaPQ',
 'Xhi01hd4YPU92OGkFKSsqEnTtVwTD_lERDpFttdWrpKSPBBZ698GvDYGkYEeq31DVz67O3oNNlEgSw',
 'HmOcVn8rE4ArEVltljRl4pWndPLIPRf9GTG3WA_vyrJXJ7fQLVGXqZ1yYVgBTXxn3-x2ACphfh9Syw',
 'VzvO1bwgi0OQCnxU96D0R-wrrCp4aM9qbA4R0lSJ9ZMPD3gRVSsfxRI5x7KUJdlY0205lG862-tywQ',
 'nQd7Ylk472T9NtwuS-hhuLDcxoUg71HK60rRIvJY1b9YMylIjeQKRVzA6p3L7RTm7OMfNISOQMR-5Q',
 '84_xB7Ms1eQbyvGb6w7SKbxPm2AtlUUfCHgqF5WrgKNhpeppB8uzrtS6osCVQ5afvmVWAcnmXZWROg',
 'a62cDaHm9nna2bWRLfiDZ2foba7f-aK663hWCJiEr8M1p9MRM9LnXO4ZBBO3j9X3oYUqvuspiKJt2w',
 '4F

In [26]:
for n, riot_puuid in enumerate(res):
    print(riot_puuid)


oFdCaHSdLE-umAd_AUTwskzbTOEPp-CU7-73Gv7wTpF2TWKw18JTx8F86EzUTMf6goBR9ZVoAwFUWA
Hd3lUYJgb7DENibojGOd_jOBjNUojDepDeCs6OLxHF80NkRdwr9UA_BJK5qQUD7h6wacRJ42sb8uPA
lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A
verjeqL6V6X4yDSQ6frKQSdh_u9OjwMfp9YJ3jetwkcNj-tyzov3zduVtY-SEVUGAQ_hz4lLE7URsA
QQJnzlpLTSBZvRvKUe4MDfJiFGAf3G-IDy0T96h7gCB5T9BN40K9tBzpg2SnFP55oxdqI8LTUQZGeg
0CpS1bKgKnH7IKuMVnpeOpyLFIj3J78S1jpxZmOx3EZq4jNp4LyIChKBAbfwpnh0of9ooKp7h8VaPQ
Xhi01hd4YPU92OGkFKSsqEnTtVwTD_lERDpFttdWrpKSPBBZ698GvDYGkYEeq31DVz67O3oNNlEgSw
HmOcVn8rE4ArEVltljRl4pWndPLIPRf9GTG3WA_vyrJXJ7fQLVGXqZ1yYVgBTXxn3-x2ACphfh9Syw
VzvO1bwgi0OQCnxU96D0R-wrrCp4aM9qbA4R0lSJ9ZMPD3gRVSsfxRI5x7KUJdlY0205lG862-tywQ
nQd7Ylk472T9NtwuS-hhuLDcxoUg71HK60rRIvJY1b9YMylIjeQKRVzA6p3L7RTm7OMfNISOQMR-5Q
84_xB7Ms1eQbyvGb6w7SKbxPm2AtlUUfCHgqF5WrgKNhpeppB8uzrtS6osCVQ5afvmVWAcnmXZWROg
a62cDaHm9nna2bWRLfiDZ2foba7f-aK663hWCJiEr8M1p9MRM9LnXO4ZBBO3j9X3oYUqvuspiKJt2w
4FdoGw1rNXZ0MlNGI3hO4bCdsDlDytyErpwmyILjPH72gweicSVj

In [10]:
puuid = 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A'

In [13]:
api_url = f"https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}?api_key={api_key}"

In [15]:
requests.get(api_url)

<Response [200]>

In [17]:
resp = requests.get(api_url)

In [19]:
mastery_info = resp.json()
mastery_info

[{'puuid': 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
  'championId': 99,
  'championLevel': 7,
  'championPoints': 470788,
  'lastPlayTime': 1702870310000,
  'championPointsSinceLastLevel': 449188,
  'championPointsUntilNextLevel': 0,
  'chestGranted': True,
  'tokensEarned': 0,
  'summonerId': 'YF1OzBQF2XhkKVV0npPVrdaJeD2R929mn0iJBphmMFlhtpUR'},
 {'puuid': 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
  'championId': 22,
  'championLevel': 7,
  'championPoints': 338152,
  'lastPlayTime': 1702434731000,
  'championPointsSinceLastLevel': 316552,
  'championPointsUntilNextLevel': 0,
  'chestGranted': True,
  'tokensEarned': 0,
  'summonerId': 'YF1OzBQF2XhkKVV0npPVrdaJeD2R929mn0iJBphmMFlhtpUR'},
 {'puuid': 'lXYH_RFUKyaBmlgvHAltJq9rOcu-UCH1_DBbVTM9QUA3ITPgHGrKmpGLbsVJoIPE1Lz1b74uR5E66A',
  'championId': 40,
  'championLevel': 7,
  'championPoints': 258065,
  'lastPlayTime': 1702183724000,
  'championPointsSinceLast

In [52]:
def commit_champ_mastery(conn, summoner_name):
    with conn.cursor() as curs:
        cur.execute ('''DROP TABLE
              "YRDEN".lol_champ_mastery''')
        cur.execute('''CREATE TABLE "YRDEN".lol_champ_mastery
                    (puuid varchar , 
                    summonerId varchar,
                    championId int , 
                    championLevel int ,
                    championPoints int ,
                    championPointsUntilNextLevel int,
                    championPointsSinceLastLevel int,
                    chestGranted boolean,
                    lastPlayTime timestamp,
                    tokensEarned int
                    )
                    ;
                    ''')
        for i, riot_puuid in enumerate(summoner_name):
            api_url = f"https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{riot_puuid}?api_key={api_key}"
            resp = requests.get(api_url)
            vals = resp.json()
            new_vals = {}
            for n in range(len(vals)):
                new_vals.update({'puuid' : vals[n]['puuid'],
                                'summonerId' : vals[n]['summonerId'],
                                 'championId' : vals[n]['championId'],
                                 'championLevel' : vals[n]['championLevel'],
                                 'championPoints' :  vals[n]['championPoints'],
                                 'championPointsUntilNextLevel' :  vals[n]['championPointsUntilNextLevel'],
                                 'championPointsSinceLastLevel' :  vals[n]['championPointsSinceLastLevel'],
                                 'chestGranted' :  vals[n]['chestGranted'],
                                 'tokensEarned' :  vals[n]['tokensEarned']
                                })

                curs.execute('''DROP TABLE "YRDEN".stage_champ_mastery;''')

                curs.execute('''CREATE TABLE "YRDEN".stage_champ_mastery
                            (puuid varchar , 
                                        summonerId varchar,
                                        championId int , 
                                        championLevel int ,
                                        championPoints int ,
                                        championPointsUntilNextLevel int,
                                        championPointsSinceLastLevel int,
                                        chestGranted boolean,
                                        tokensEarned int
                                        );''')
                #print(new_vals)

                curs.execute('''INSERT INTO "YRDEN".stage_champ_mastery 
                            (puuid, 
                            summonerId,
                            championId,
                            championLevel,
                            championPoints,
                            championPointsUntilNextLevel,
                            championPointsSinceLastLevel,
                            chestGranted,
                           tokensEarned
                            )
                                VALUES (%(puuid)s, %(summonerId)s, %(championId)s,
                                %(championLevel)s, %(championPoints)s,
                                %(championPointsUntilNextLevel)s,
                                %(championPointsSinceLastLevel)s, 
                                %(chestGranted)s,   %(tokensEarned)s);''', new_vals)



                curs.execute('''INSERT INTO "YRDEN".lol_champ_mastery (puuid, 
                            summonerId,
                            championId,
                            championLevel,
                            championPoints,
                            championPointsUntilNextLevel,
                            championPointsSinceLastLevel,
                            chestGranted,
                            tokensEarned
                            )
                                SELECT DISTINCT cmst.puuid, 
                            cmst.summonerId,
                            cmst.championId,
                            cmst.championLevel,
                            cmst.championPoints,
                            cmst.championPointsUntilNextLevel,
                            cmst.championPointsSinceLastLevel,
                            cmst.chestGranted,
                            cmst.tokensEarned
                                FROM "YRDEN".stage_champ_mastery cmst
                                WHERE NOT EXISTS (SELECT puuid,
                                summonerId, championId
                                FROM "YRDEN".lol_champ_mastery cmt
                                WHERE cmst.puuid = cmt.puuid
                                AND cmst.summonerId = cmt.summonerId
                                AND cmst.championId = cmt.championId);
                                 ''')
                            
                conn.commit()
            curs.execute('''SELECT DISTINCT CHAMPIONID FROM "YRDEN".LOL_CHAMP_MASTERY WHERE PUUID = %s;''', (riot_puuid,))
            print(curs.rowcount, " champions added for ", i)

In [53]:
cur.execute('SELECT riot_puuid FROM "YRDEN".yrden_people WHERE summoner_name is NOT NULL AND riot_puuid is NOT NULL')
result = cur.fetchall()
res = [result[0] for result in result]
res
commit_champ_mastery(conn, res)

161  champions added for  0
134  champions added for  1
162  champions added for  2
163  champions added for  3
144  champions added for  4
164  champions added for  5
138  champions added for  6
158  champions added for  7
153  champions added for  8
141  champions added for  9
165  champions added for  10
162  champions added for  11
147  champions added for  12
110  champions added for  13


In [46]:
conn.close()

In [ ]:
#
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDItem = DDRegion['item']

In [ ]:
DDRegion

In [ ]:
lol_watcher.data_dragon.items(DDItem)